In [227]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds #To import mnist dataset

# Data

In [228]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist',with_info = True, as_supervised = True)
# with_info=True provides us with a tuple which contains information about the version, features and number of samples
# as_supervised=True loads the dataset in a 2-tuple structure (input, target) 


In [229]:
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']

# firstly we define the number of validation samples as a % of the train samples (10%)
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64) #To ensure its an integer value

# We also store the number of test samples in a variable
num_test_samples = mnist_info.splits['train'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64) #Again, to ensure its an integer value

def scale(image,label):
    image = tf.cast(image,tf.float32) #Cast the tensor into a float
    image = image/255. #Scale values between 0 and 1 that is why we divide by 255. The dot is because we want a float value
    return image,label

# the method .map() allows us to apply a custom transformation to a given dataset
scaled_train_and_validation_data = mnist_train.map(scale) #.map(function "scale" defined above)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000 #In order to shuffle in chunks size that we select

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

#we create a batch with a batch size equal to the total number of validation samples
#.take() function returns elements from array along the mentioned axis and indices.
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

#similarly for the train_data...
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 1000

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples) #a batch size 1
test_data = test_data.batch(num_test_samples)

# takes next batch (it is the only batch)
# because as_supervized=True, we've got a 2-tuple structure
validation_inputs, validation_targets = next(iter(validation_data))
#next loads the next batch
#iter creates an object which can be iterated one element at a time


# Model

In [230]:
input_size = 784
output_size = 10
hidden_layer_size = 500

In [231]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)), 
    #Flatten converts our tensor (28x28x1) into a vector (784), being this our input layer.
    
    # tf.keras.layers.Dense is in essence: output = activation function(dot(input, weight) + bias)
    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation = 'tanh'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
    
])

## Choose the optimizer and the loss function

In [232]:
#custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.02)
model.compile(optimizer ='adam',loss='sparse_categorical_crossentropy', metrics= ['accuracy']) #sparse... applies one-hot encoding

# Training

In [233]:
NUM_EPOCHS = 5

#Now we fit the model specifying the training data, the number of epochs and the validation data
model.fit(train_data, epochs = NUM_EPOCHS, 
          validation_data =(validation_inputs,validation_targets),verbose = 2)

Epoch 1/5
54/54 - 10s - loss: 0.5469 - accuracy: 0.8204 - val_loss: 0.1691 - val_accuracy: 0.9525
Epoch 2/5
54/54 - 9s - loss: 0.1260 - accuracy: 0.9616 - val_loss: 0.1148 - val_accuracy: 0.9655
Epoch 3/5
54/54 - 9s - loss: 0.0850 - accuracy: 0.9743 - val_loss: 0.0856 - val_accuracy: 0.9745
Epoch 4/5
54/54 - 9s - loss: 0.0615 - accuracy: 0.9808 - val_loss: 0.0713 - val_accuracy: 0.9780
Epoch 5/5
54/54 - 9s - loss: 0.0441 - accuracy: 0.9870 - val_loss: 0.0569 - val_accuracy: 0.9822


# Model Test

In [234]:
test_loss,test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.0949 - accuracy: 0.9741


In [235]:
print('Test loss:{0:.2f}. Test accuracy:{1:.2f}%'.format(test_loss,test_accuracy*100.))

Test loss:0.09. Test accuracy:97.41%
